# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（関数名・引数名・返却列・順序）
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `RequestAccepted` から **友だち数が最大の人**とその**友だち数**を求める（フレンドは無向）
* 入力 DF: `RequestAccepted(requester_id:int, accepter_id:int, accept_date:date)`
* 出力: `id:int, num:int`

  * 同率最大が複数いても全員返す（順序は任意）

## 2) 実装（指定シグネチャ厳守）

> 列を最小化し、**両端点を縦に結合 → 頻度集計**で次数を得て、**最大値のみ抽出**します。`value_counts(sort=False)` を使い、不要な並び替えは発生させません。

```python
import pandas as pd
import numpy as np

def solve_request_accepted(request_accepted: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['id', 'num']
    """
    # 空入力対策（念のため）
    if request_accepted.empty:
        return pd.DataFrame(columns=['id', 'num'])

    # 必要列のみ（I/Oなし・並び替えなし）
    df = request_accepted[['requester_id', 'accepter_id']]

    # 両端点を縦結合して次数をカウント（無向化）
    # sort=False で不要な並び替えを抑止
    deg = pd.concat([df['requester_id'], df['accepter_id']], ignore_index=True).value_counts(sort=False)

    # 最大次数を抽出し、その次数を持つユーザーを全員返す
    max_deg = int(deg.max())
    winners = deg.index[deg.eq(max_deg)]

    out = pd.DataFrame({
        'id': pd.Series(winners, dtype='int64').to_numpy(),
        'num': np.full(len(winners), max_deg, dtype='int64'),
    })

    return out

Analyze Complexity
Runtime 2590 ms
Beats 90.50%
Memory 66.99 MB
Beats 81.54%

```

> **実運用の堅牢化（任意）**
> データに `(a,b)` と `(b,a)` の**両方向エッジが共存**し得る場合は、下記のように無向正規化（`np.minimum/maximum`）→ `drop_duplicates` → 同手順でカウントすると二重カウントを防げます。
>
> ```python
> uv = pd.DataFrame({
>     'u': np.minimum(df['requester_id'].to_numpy(), df['accepter_id'].to_numpy()),
>     'v': np.maximum(df['requester_id'].to_numpy(), df['accepter_id'].to_numpy()),
> }).drop_duplicates()
> deg = pd.concat([uv['u'], uv['v']], ignore_index=True).value_counts(sort=False)
> ```

## 3) アルゴリズム説明

* 使用 API

  * `pd.concat`：2 列を縦結合して端点列を作成
  * `Series.value_counts(sort=False)`：各ユーザーの頻度（= 友だち数）を算出
  * ブール条件で最大値の要素だけを抽出
* **NULL / 重複 / 型**

  * 入力は整数想定。欠損があり得る場合は `dropna()` を結合前に追加
  * 対称重複（`(a,b)` と `(b,a)`）を一意化するには上記の「堅牢化」案を使用
  * 返却列は `int64` に明示

## 4) 計算量（概算）

* 縦結合：**O(N)**
* `value_counts`（ハッシュ集計想定）：**O(N)** 近辺
* 最大値抽出：**O(U)**（`U` はユニークユーザー数）

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 データフレーム RequestAccepted] --> B[必要列のみ抽出 requester_id accepter_id]
  B --> C[縦結合で端点列作成]
  C --> D[value_counts で次数集計]
  D --> E[最大次数へフィルタ]
  E --> F[出力 id num]
```
